In [754]:
import torch
import numpy as np
import torch.nn.functional as F
from torchvision.transforms.functional import adjust_contrast
a= torch.rand(64,64, 3) # N H W C
input = torch.stack([a,a])
parameters = torch.Tensor([0.5,0.5])

In [366]:
def relu(x):
    x_ = x.copy()
    x_[x_<0] = 0
    return x_

class AdjustContraste():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, list_editted, parameters):     
        editted  =  list_editted.numpy()
        mean = editted.mean()
        editted_ = (editted-mean)*(parameters[0]+1)+mean
        editted_ = relu(editted_)
        editted_ = 1-relu(1-editted_)
        return [editted_]
    
old = AdjustContraste()(a,[0.5])[0]

In [367]:
class AdjustContrast():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, images:torch.Tensor, parameters:torch.Tensor):
        batch_size = parameters.shape[0]
        mean = images.view(batch_size,-1).mean(1)
        mean = mean.view(batch_size, 1, 1, 1)
        parameters = parameters.view(batch_size, 1, 1, 1)
        editted = (images-mean)*(parameters+1)+mean
        editted = F.relu(editted)
        editted = 1-F.relu(1-editted)
        return editted
new = AdjustContrast()(input,torch.Tensor([0.5,0.5]))

In [368]:
(old==new[0].numpy()).all()

np.True_

In [369]:
from envs.dehaze.src import dehaze

class AdjustDehazee():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["dehaze"]

    def __call__(self, list_editted, parameters):
        editted = list_editted.numpy()
        scale = max((editted.shape[:2])) / 512.0
        omega = parameters[0]
        editted_ = dehaze.DarkPriorChannelDehaze(
            wsize=int(15*scale), radius=int(80*scale), omega=omega,
            t_min=0.25, refine=True)(editted * 255.0) / 255.0
        editted_ = relu(editted_)
        editted_ = 1-relu(1-editted_)
        return [editted_]
old = AdjustDehazee()(a,[0.5])[0]

In [399]:
class AdjustDehaze():

    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["dehaze"]

    def __call__(self, images, parameters):
        """
        Takes a batch of images where B (the last dim) is the batch size
        args:
            images: torch.Tensor # B H W C 
            parameters :torch.Tensor # N
        return:
            output: torch.Tensor #  B H W C 
        """
        assert images.dim()==4
        batch_size = parameters.shape[0]
        output = []
        for image_index in range(batch_size):
            image = images[image_index].numpy()
            scale = max((image.shape[:2])) / 512.0
            omega = float(parameters[image_index])
            editted= dehaze.DarkPriorChannelDehaze(
                wsize=int(15*scale), radius=int(80*scale), omega=omega,
                t_min=0.25, refine=True)(image * 255.0) / 255.0
            editted = torch.tensor(editted)
            editted = F.relu(editted)
            editted= 1-F.relu(1-editted)
            output.append(editted)
        output = torch.stack(output)
        return output
    
    
new = AdjustDehaze()(input,parameters)

In [371]:
(old==new.numpy()[0]).all()

np.True_

In [448]:
import cv2

class AdjustClaritye():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["clarity"]

    def __call__(self, list_editted, parameters):
        editted = list_editted.numpy()
        scale = max((editted.shape[:2])) / 512.0
        clarity = parameters[0]

        unsharped = cv2.bilateralFilter((editted*255.0).astype(np.uint8),
                                            int(32*scale), 50, 10*scale)/255.0
        editted_ = editted + (editted-unsharped) * clarity
        return [editted_]
    
old = AdjustClaritye()(a,[0.5])[0]

In [453]:

class AdjustClarity():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["clarity"]

    def __call__(self, images, parameters):
        """
        Takes a batch of images where B (the last dim) is the batch size
        args:
            images: torch.Tensor # B H W C 
            parameters :torch.Tensor # N
        return:
            output: torch.Tensor #  B H W C 
        """
        assert images.dim()==4
        batch_size = parameters.shape[0]
        output = [] 
        clarity = parameters.view(batch_size, 1, 1, 1)
        for image in images: 
            input = image.numpy()      
            scale = max((input.shape[:2])) / 512.0
            unsharped = cv2.bilateralFilter((input*255.0).astype(np.uint8),
                                                int(32*scale), 50, 10*scale)/255.0
            output.append(torch.tensor(unsharped))
        output = torch.stack(output) 
        editted_images = images + (images-output) * clarity
        
        return editted_images
    
# class AdjustClarity():
#     def __init__(self):
#         self.num_parameters = 1
#         self.window_names = ["parameter"]
#         self.slider_names = ["clarity"]

#     def __call__(self, images, parameters):
#         assert images.dim()==4
#         batch_size = parameters.shape[0]
#         output = [] 
        
#         for image_index,image in enumerate(images): 
#             clarity = float(parameters[image_index])
#             input = image.numpy()      
#             scale = max((input.shape[:2])) / 512.0
#             unsharped = cv2.bilateralFilter((input*255.0).astype(np.uint8),
#                                                 int(32*scale), 50, 10*scale)/255.0
#             editted = input + (input-unsharped) * clarity
#             output.append(torch.tensor(editted))

#         output = torch.stack(output) 

#         return output
new = AdjustClarity()(input,parameters)

In [455]:
(old==new.numpy()[1]).all()

np.True_

# exposure

In [636]:
def sigmoid_inverse(y):
    epsilon = 10**(-3)
    y = F.relu(y-epsilon)+epsilon
    y = 1-epsilon-F.relu((1-epsilon)-y)
    y = (1/y)-1
    output = -np.log(y.numpy())
    return torch.tensor(output)

class SigmoidInverse():

    def __init__(self):
        self.num_parameters = 0

    def __call__(self, images):
        return sigmoid_inverse(images)
new_sig_inv = SigmoidInverse()

def old_sigmoid_inverse(y):
    epsilon = 10**(-3)
    y_ = y.copy()
    y_ = relu(y_-epsilon)+epsilon
    y_ = 1-epsilon-relu((1-epsilon)-y_)
    y_ = (1/y_)-1
    output = -np.log(y_)
    return output

In [637]:
(old_sigmoid_inverse(a.numpy())==new_sig_inv (input)[0].numpy()).all()

np.True_

In [639]:
class AdjustExposuree():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["exposure"]

    def __call__(self, list_sigmoid_inversed, parameters):
        exposure = parameters[0]
        return [old_sigmoid_inverse(list_sigmoid_inversed)+ exposure*5]
old = AdjustExposuree()(a.numpy(),[0.5])

In [640]:
class AdjustExposure():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["exposure"]

    def __call__(self, images, parameters):
        batch_size = parameters.shape[0]
        exposure = parameters.view(batch_size, 1, 1, 1)
        output = images+exposure*5
        output = new_sig_inv(output)
        return output
new = AdjustExposure()(input,parameters)

In [641]:
(old_sigmoid_inverse(a.numpy())==new_sig_inv (input)[0].numpy()).all()

np.True_

# Temp

In [770]:
class AdjustTempe():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["temp"]

    def __call__(self, list_sigmoid_inversed, parameters):
        temp = parameters[0]
        sigmoid_inversed_ = list_sigmoid_inversed.copy()
        if temp > 0:
            sigmoid_inversed_[:,:,1] += temp*1.6
            sigmoid_inversed_[:,:,2] += temp*2
        else:
            sigmoid_inversed_[:,:,0] -= temp*2.0
            sigmoid_inversed_[:,:,1] -= temp*1.0
        return [sigmoid_inversed_]
old = AdjustTempe()(a.numpy(),[0.5])[0]

In [773]:
class AdjustTemp():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["temp"]

    def __call__(self, images, parameters):
        batch_size = parameters.shape[0]
        temp = parameters.view(batch_size, 1, 1, 1)
        editted = torch.clone(images)  

        index_high = (temp>0).view(-1)
        index_low = (temp<=0).view(-1)

        editted[index_high,:,:,1] += temp[index_high,:,:,0]*1.6
        editted[index_high,:,:,2] += temp[index_high,:,:,0]*2   
        editted[index_low,:,:,0] -= temp[index_low,:,:,0]*2.0
        editted[index_low,:,:,1] -= temp[index_low,:,:,0]*1.0          

        return editted
    
new = AdjustTemp()(input,parameters)

In [774]:
(old==new.numpy()[1]).all()

np.True_

# Tint

In [807]:
class AdjustTinte():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["tint"]

    def __call__(self, list_sigmoid_inversed, parameters):
        tint = parameters[0]
        sigmoid_inversed_ = list_sigmoid_inversed.copy()
        if tint > 0:
            sigmoid_inversed_[:,:,0] += tint*2
            sigmoid_inversed_[:,:,2] += tint*1
        else:
            sigmoid_inversed_[:,:,1] -= tint*2
            sigmoid_inversed_[:,:,2] -= tint*1
        return [sigmoid_inversed_]
    
old = AdjustTinte()(a.numpy(),[0.5])[0]

In [808]:
class AdjustTint():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["tint"]

    def __call__(self, images, parameters):
        batch_size = parameters.shape[0]
        tint = parameters.view(batch_size, 1, 1, 1)
        editted = torch.clone(images)  

        index_high = (tint>0).view(-1)
        index_low = (tint<=0).view(-1)

        editted[index_high,:,:,0] += tint[index_high,:,:,0]*2
        editted[index_high,:,:,2] += tint[index_high,:,:,0]*1  
        editted[index_low,:,:,1] -= tint[index_low,:,:,0]*2
        editted[index_low,:,:,2] -= tint[index_low,:,:,0]*1         

        return editted
    
new = AdjustTint()(input,parameters)

In [809]:
(old==new.numpy()[1]).all()

np.True_

# BGR2HSV

In [1144]:
class Bgr2Hsve():
    def __init__(self):
        self.num_parameters = 0

    def __call__(self, list_editted, parameters):
        editted = list_editted

        max_bgr = editted.max(axis=2)
        min_bgr = editted.min(axis=2)

        b_g = editted[:,:,0]-editted[:,:,1]
        g_r = editted[:,:,1]-editted[:,:,2]
        r_b = editted[:,:,2]-editted[:,:,0]

        b_min_flg = (1-relu(np.sign(b_g)))*relu(np.sign(r_b))
        g_min_flg = (1-relu(np.sign(g_r)))*relu(np.sign(b_g))
        r_min_flg = (1-relu(np.sign(r_b)))*relu(np.sign(g_r))

        epsilon = 10**(-5)
        h1 = 60*g_r/(max_bgr-min_bgr+epsilon)+60
        h2 = 60*b_g/(max_bgr-min_bgr+epsilon)+180
        h3 = 60*r_b/(max_bgr-min_bgr+epsilon)+300
        h = h1*b_min_flg + h2*r_min_flg + h3*g_min_flg

        v = max_bgr
        s = (max_bgr-min_bgr)/(max_bgr+epsilon)

        return [h,s,v]
old = Bgr2Hsve()(a.numpy(),[0.5])

In [1145]:
class Bgr2Hsv:
    def __init__(self):
        self.num_parameters = 0

    def __call__(self, images ,parameters=None):
        editted = images

        max_bgr, _ = editted.max(dim=-1, keepdim=True)
        min_bgr, _ = editted.min(dim=-1, keepdim=True)

        b = editted[..., 0]
        g = editted[..., 1]
        r = editted[..., 2]

        b_g = b - g
        g_r = g - r
        r_b = r - b

        b_min_flg = (1 - F.relu(torch.sign(b_g))) * F.relu(torch.sign(r_b))
        g_min_flg = (1 - F.relu(torch.sign(g_r))) * F.relu(torch.sign(b_g))
        r_min_flg = (1 - F.relu(torch.sign(r_b))) * F.relu(torch.sign(g_r))

        epsilon = 10**(-5)
        h1 = 60 * g_r / (max_bgr.squeeze() - min_bgr.squeeze() + epsilon) + 60
        h2 = 60 * b_g / (max_bgr.squeeze() - min_bgr.squeeze() + epsilon) + 180
        h3 = 60 * r_b / (max_bgr.squeeze() - min_bgr.squeeze() + epsilon) + 300
        h = h1 * b_min_flg + h2 * r_min_flg + h3 * g_min_flg

        v = max_bgr.squeeze()
        s = (max_bgr.squeeze() - min_bgr.squeeze()) / (max_bgr.squeeze() + epsilon)

        return [h, s, v]
    
new = Bgr2Hsv()(input,parameters)

In [1146]:
assert (old[1]==new[1].numpy()).all()
assert (old[2]==new[2].numpy()).all()
assert (old[0]==new[0].numpy()).all()

# Shadows

In [1147]:
h,s,v = torch.clone(new[0]),torch.clone(new[1]),torch.clone(new[2])

In [1158]:
def numpy_sigmoid(x):
    return 1/(1+np.exp(-x))

class AdjustShadowse():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["shadows"]

    def __call__(self, list_hsv, parameters):
        shadows = parameters[0]
        v = list_hsv[2]
        shadows_mask = 1-numpy_sigmoid((v-0)*5.0)
        return [list_hsv[0], list_hsv[1], v*(1+shadows_mask*shadows*5.0)],shadows_mask
    
old,o = AdjustShadowse()([h.numpy(),s.numpy(),v.numpy()],[0.5])

In [1159]:
class AdjustShadows:
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["shadows"]
    
    def __call__(self, list_hsv, parameters):
        batch_size = parameters.shape[0]
        shadows = parameters.view(batch_size, 1, 1).numpy()

        v = list_hsv[2].numpy()
        
        # Calculate shadows mask

        shadows_mask = 1 - numpy_sigmoid((v - 0.0) * 5.0)
        # Adjust v channel based on shadows mask
        adjusted_v = v * (1 + shadows_mask * shadows * 5.0)
        adjusted_v = torch.tensor(adjusted_v)
        return [list_hsv[0], list_hsv[1], adjusted_v],shadows_mask

new,n = AdjustShadows()([h,s,v],parameters)

In [1160]:
assert (old[0]==new[0].numpy()).all()
assert (old[1]==new[1].numpy()).all()
assert (old[2]==new[2].numpy()).all()